In [1]:
pip install scikit-learn


   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   --- ------------------------------------ 1.0/10.7 MB 7.2 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/10.7 MB 5.0 MB/s eta 0:00:02
   -------- ------------------------------- 2.4/10.7 MB 4.8 MB/s eta 0:00:02
   ------------ --------------------------- 3.4/10.7 MB 4.3 MB/s eta 0:00:02
   --------------- ------------------------ 4.2/10.7 MB 4.2 MB/s eta 0:00:02
   ------------------- -------------------- 5.2/10.7 MB 4.1 MB/s eta 0:00:02
   ---------------------- ----------------- 6.0/10.7 MB 4.1 MB/s eta 0:00:02
   ------------------------- -------------- 6.8/10.7 MB 4.1 MB/s eta 0:00:01
   ---------------------------- ----------- 7.6/10.7 MB 4.1 MB/s eta 0:00:01
   ------------------------------- -------- 8.4/10.7 MB 4.0 MB/s eta 0:00:01
   ---------------------------------- ----- 9.2/10.7 MB 4.0 MB/s eta 0:00:01
   ----------------------------------- ---- 9.4/10.7 MB 3.8 MB/s eta 0:00:01
   ---

In [3]:
import pandas as pd
import mysql.connector
import matplotlib.pyplot as plt
import seaborn as sns

# Set up MySQL connection
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Suku@#0508',
    database='infrasense_ai'
)



Model 1 – Predict CPU Overload

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load server logs
df = pd.read_sql("SELECT * FROM server_logs", conn)

# Feature engineering
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hour'] = df['timestamp'].dt.hour
df['is_overloaded'] = (df['cpu_usage'] > 85).astype(int)

features = ['ram_usage', 'disk_io', 'net_usage', 'hour']
X = df[features]
y = df['is_overloaded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


C:\Users\shash\AppData\Local\Temp\ipykernel_21008\1259423429.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM server_logs", conn)


Train Model

In [5]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.95      1.00      0.97        76
           1       0.00      0.00      0.00         4

    accuracy                           0.95        80
   macro avg       0.47      0.50      0.49        80
weighted avg       0.90      0.95      0.93        80

Accuracy: 0.95


c:\Users\shash\Desktop\InfraSense-AI\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\shash\Desktop\InfraSense-AI\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\shash\Desktop\InfraSense-AI\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize

Model 2 – Predict Helpdesk Resolution Time

In [6]:
#Load Helpdesk Data
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df = pd.read_sql("SELECT * FROM helpdesk_tickets", conn)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Simulate resolution time
import numpy as np
np.random.seed(42)
df['resolution_time'] = np.random.normal(loc=5, scale=2, size=len(df))  # hours

# Encode priority
df['priority_code'] = df['priority'].map({'High': 3, 'Medium': 2, 'Low': 1})

X = df[['priority_code']]
y = df['resolution_time']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


C:\Users\shash\AppData\Local\Temp\ipykernel_21008\2219232604.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM helpdesk_tickets", conn)


Train Regression Model

In [7]:
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))
print("Sample prediction:", reg.predict([[3]]))  # High priority


MSE: 2.5012604228468596
Sample prediction: [4.8581271]


c:\Users\shash\Desktop\InfraSense-AI\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
